# Import the Necessary Libraries

In [1]:
import torch 
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchsummary import summary # pip install torch-summary
from tqdm import tqdm 

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cpu


In [3]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Download the CIFAR 10 Dataset

In [44]:
# Get the training data and create a transform to ensure the data is made up of tensors
# why are we not normalizing the data between 0 and 1?
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform=transform)

# load the training data and transform it
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

# Get the test data  
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform=transform)

# Now load the test data with a batch size of 32: Do we need to shuffle the test data? 
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2) #None # write your code here

Files already downloaded and verified
Files already downloaded and verified


# Building the Model

In [34]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # Create instance variables for the layers you will pass in the forward method
        input_size = 32*32*3
        self.dense_layer1 = torch.nn.Linear(input_size, 200)  # First dense layer with 200 neurons
        self.dense_layer2 = torch.nn.Linear(200, 150)         # Second dense layer with 150 neurons
        self.dense_layer3 = torch.nn.Linear(150, 10)          # Output layer with 10 neurons

    def forward(self, x):
        x = x.view(-1, self.num_flat_features(x))  # Flatten the input
        x = torch.nn.functional.relu(self.dense_layer1(x))                   # Apply ReLU activation to first layer
        x = torch.nn.functional.relu(self.dense_layer2(x))                   # Apply ReLU activation to second layer
        x = torch.nn.functional.softmax(self.dense_layer3(x), dim=1)         # Apply softmax activation to output layer
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # Get dimensions excluding batch size
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Example usage:


mlp_model = MLP()

# Set the Loss Function and Optimizer

In [35]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.0005)

# Write the Train Function

In [42]:
def train(dataloader, model, criterion, optimizer):
    model.train()
    batch_bar = tqdm(total=len(dataloader), position=0, leave=False, dynamic_ncols=True, desc='Train')
    total_loss = 0
    total_accuracy = 0

    for i, (imgs, labels) in enumerate(dataloader):
        # write your code below for the train function 
        # Forward pass
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / labels.size(0)

        # Update total loss and accuracy
        total_loss += loss.item()
        total_accuracy += correct

        # Update the progress bar
        batch_bar.update()
    batch_bar.close()
    
    return total_loss/len(dataloader), 100*total_accuracy/(len(dataloader)*32)

## Write the Evaluate Function to get the Test Accuracy

In [40]:
def evaluate(dataloader, model):
    model.eval()
    # write your code for the evaluate function
    #raise NotImplemented
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient tracking during evaluation
        for imgs, labels in dataloader:
            # Forward pass
            outputs = model(imgs)
            
            # Compute loss
            loss = criterion(outputs, labels)
            
            # Compute the number of correct predictions
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            
            # Update total loss and total samples
            total_loss += loss.item()
            total_samples += labels.size(0)

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples

    return avg_loss, accuracy

# Train the MLP

In [43]:
for epoch in range(0, 10):
    train_loss, train_acc = train(train_dataloader, mlp_model, criterion, optimizer)
    print(f"Epoch {epoch+1}/{10} - Train Loss: {train_loss:.4f} - Train Accuracy: {train_acc:.4f}")

Epoch 1/10 - Train Loss: 2.0109 - Train Accuracy: 44.5078


Epoch 2/10 - Train Loss: 2.0081 - Train Accuracy: 44.8097


Epoch 3/10 - Train Loss: 2.0006 - Train Accuracy: 45.5294


Epoch 4/10 - Train Loss: 1.9946 - Train Accuracy: 46.2292


Epoch 5/10 - Train Loss: 1.9927 - Train Accuracy: 46.4032


Epoch 6/10 - Train Loss: 1.9897 - Train Accuracy: 46.8030


Epoch 7/10 - Train Loss: 1.9864 - Train Accuracy: 47.0789


Epoch 8/10 - Train Loss: 1.9810 - Train Accuracy: 47.6368


Epoch 9/10 - Train Loss: 1.9781 - Train Accuracy: 47.9027


Epoch 10/10 - Train Loss: 1.9737 - Train Accuracy: 48.4165


# Get Your Test Accuracy

In [45]:
test_acc = evaluate(test_dataloader, mlp_model)
test_acc

(1.9952165021683081, 0.4613)